# Run A1111 in runpod
# NoteBook Created by Voiid (https://github.com/official-imvoiid)

In [ ]:
# Setup A1111

!python -m pip install --upgrade pip
!pip install insightface

import os
import subprocess
import sys

# Config
base_dir = "/workspace"  # RunPod typically uses /workspace
webui_dir = os.path.join(base_dir, "stable-diffusion-webui")
extensions = [
    "https://github.com/Mikubill/sd-webui-controlnet",
    "https://github.com/deforum-art/deforum-for-automatic1111-webui",
    "https://github.com/camenduru/stable-diffusion-webui-images-browser",
    "https://github.com/Bing-su/adetailer.git",
    "https://github.com/hako-mikan/sd-webui-regional-prompter",
    "https://github.com/AUTOMATIC1111/stable-diffusion-webui-promptgen",
    "https://github.com/BlafKing/sd-civitai-browser-plus",
    "https://github.com/dan4in/PonyHelper"
]

def run_command(command, cwd=None):
    """Run shell command and handle errors"""
    try:
        print(f"Running: {command}")
        result = subprocess.run(
            command,
            shell=True,
            check=True,
            cwd=cwd,
            capture_output=True,
            text=True
        )
        if result.stdout:
            print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {command}")
        print(f"Error: {e.stderr}")
        return False

def main():
    print("Starting Automatic1111 WebUI setup for RunPod...")
    
    # Create base directory if it doesn't exist
    os.makedirs(base_dir, exist_ok=True)
    
    # Clone Automatic1111 if not present
    if not os.path.exists(webui_dir):
        print("Cloning Automatic1111 WebUI...")
        if not run_command(f"git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui {webui_dir}"):
            print("Failed to clone Automatic1111 WebUI")
            sys.exit(1)
        print("✓ Automatic1111 WebUI cloned successfully")
    else:
        print("✓ Automatic1111 WebUI already exists")
    
    # Create extensions directory if it doesn't exist
    extensions_dir = os.path.join(webui_dir, "extensions")
    os.makedirs(extensions_dir, exist_ok=True)
    
    # Clone each extension
    print(f"\nInstalling {len(extensions)} extensions...")
    success_count = 0
    
    for ext in extensions:
        ext_name = ext.rstrip("/").split("/")[-1]
        # Remove .git suffix if present
        if ext_name.endswith(".git"):
            ext_name = ext_name[:-4]
            
        ext_path = os.path.join(extensions_dir, ext_name)
        
        if not os.path.exists(ext_path):
            print(f"\nInstalling extension: {ext_name}")
            if run_command(f"git clone {ext}", cwd=extensions_dir):
                print(f"✓ {ext_name} installed successfully")
                success_count += 1
            else:
                print(f"✗ Failed to install {ext_name}")
        else:
            print(f"✓ {ext_name} already exists")
            success_count += 1
    
    print(f"\n{'='*50}")
    print(f"Setup completed: {success_count}/{len(extensions)} extensions installed")
    print(f"WebUI location: {webui_dir}")
    print(f"{'='*50}")
    
    print("\nTo start the WebUI, run:")
    print(f"cd {webui_dir}")
    print("python launch.py --listen --enable-insecure-extension-access")

if __name__ == "__main__":
    main()

In [ ]:
# CivitAi Model Downloader Via API

import os
import re
import sys
import requests
from tqdm import tqdm

# Set your CivitAI API key here
API_KEY = "API_KEY"  # ← set your CivitAI key

# ─── Paste your URLs here ────────────────────────────────────────────────
# Base directory paths
CKPT_PATH = "/workspace/stable-diffusion-webui/models/Stable-diffusion"
LORA_PATH = "/workspace/stable-diffusion-webui/models/Lora"
VAE_PATH = "/workspace/stable-diffusion-webui/models/VAE"
CONTROLNET_PATH = "/workspace/stable-diffusion-webui/models/ControlNet"
EMBEDDING_PATH = "/workspace/stable-diffusion-webui/embeddings"

ckpt_links = [
    "https://civitai.com/models/372465?modelVersionId=914390",
    "https://civitai.com/models/439889?modelVersionId=1199750"
]

lora_links = [
    "https://civitai.com/models/696418?modelVersionId=779327",
    "https://civitai.com/models/1327644?modelVersionId=1498957",
    "https://civitai.com/models/1333749/add-detail-slider",
    "https://civitai.com/models/669571/pony-add-more-details",
    "https://civitai.com/models/454172?modelVersionId=505635",
    "https://civitai.com/models/300005?modelVersionId=436219",
    "https://civitai.com/models/73756?modelVersionId=703107"
]

vae_links = [
    "https://civitai.com/models/199948?modelVersionId=224985"
]

embedding_links = [
    "https://civitai.com/models/118418?modelVersionId=134583",
    "https://civitai.com/models/583583?modelVersionId=651084"
]

controlnet_links = [
    # add ControlNet URLs here...
]

# ──────────────────────────────────────────────────────────────────────────
# Check if API key is set
if not API_KEY:
    print("API key not set. Please set your CivitAI API key.")
    sys.exit(1)

# Create all required directories
os.makedirs(CKPT_PATH, exist_ok=True)
os.makedirs(LORA_PATH, exist_ok=True)
os.makedirs(VAE_PATH, exist_ok=True)
os.makedirs(CONTROLNET_PATH, exist_ok=True)
os.makedirs(EMBEDDING_PATH, exist_ok=True)

def parse_mid_and_override(url):
    m = re.search(r"models/(\d+)", url)
    if not m:
        raise ValueError(f"Bad URL: {url}")
    mid = m.group(1)
    over = re.search(r"modelVersionId=(\d+)", url)
    return mid, (over.group(1) if over else None)

def get_vid(mid):
    h = {"Authorization": f"Bearer {API_KEY}"}
    r = requests.get(
        f"https://civitai.com/api/v1/models/{mid}", headers=h, timeout=10
    )
    r.raise_for_status()
    return r.json()["modelVersions"][0]["id"]

def fetch_dl_info(vid):
    h = {"Authorization": f"Bearer {API_KEY}"}
    r = requests.get(
        f"https://civitai.com/api/v1/model-versions/{vid}", headers=h, timeout=10
    )
    r.raise_for_status()
    j = r.json()
    return j["downloadUrl"], j["files"][0]["name"]

def download(url, dest_folder):
    try:
        mid, override = parse_mid_and_override(url)
        vid = override or get_vid(mid)
        dl_url, fname = fetch_dl_info(vid)
        print(f"Processing: {url}")
        print(f"Model ID: {mid}, Version ID: {vid}")
        print(f"Filename: {fname}")
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return

    path = os.path.join(dest_folder, fname)

    if os.path.exists(path):
        print(f"File already exists: {path}")
        return

    print(f"Downloading to: {path}")

    try:
        r = requests.get(
            dl_url,
            headers={"Authorization": f"Bearer {API_KEY}"},
            stream=True
        )

        if r.status_code != 200:
            print(f"Download failed with status code {r.status_code}")
            return

        total = int(r.headers.get("content-length", 0))
        bar = tqdm(total=total, unit="iB", unit_scale=True, desc=fname)

        with open(path, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
                bar.update(len(chunk))

        bar.close()
        print(f"Downloaded: {fname}")
    except Exception as e:
        print(f"Error downloading {fname}: {e}")
        if os.path.exists(path):
            os.remove(path)  # Remove partially downloaded file

# ─── Kick off downloads ───────────────────────────────────────────────────
print("Starting downloads...")

print("\nDownloading checkpoint models...")
for u in ckpt_links:
    download(u, CKPT_PATH)

print("\nDownloading LoRA models...")
for u in lora_links:
    download(u, LORA_PATH)

print("\nDownloading VAE models...")
for u in vae_links:
    download(u, VAE_PATH)

print("\nDownloading embedding models...")
for u in embedding_links:
    download(u, EMBEDDING_PATH)

print("\nDownloading ControlNet models...")
for u in controlnet_links:
    download(u, CONTROLNET_PATH)

print("\n✨ All downloads completed.")

📌 Recommended Setup:
- GPU: A6000 Ada  
- Storage: 70GB total  
  - 50GB on persistent  
  - 20GB on temporary

In [ ]:
# Launch A1111 Webui

!python stable-diffusion-webui/launch.py --share --enable-insecure-extension-access